In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocess dataset as before.

In [2]:
#Drop columns
application_df = application_df.drop("EIN", axis =1)
application_df =application_df.drop("NAME", axis =1)


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE 
replace_application =list(app_type_counts[app_type_counts < 500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2380        1
C2570        1
C4200        1
C1580        1
C1820        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE
replace_class = list(classification_counts[classification_counts<1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list

encode_df =pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [9]:
# Merge one-hot encoded features and drop the originals
app_df=application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = app_df["IS_SUCCESSFUL"].values
X =app_df.drop(["IS_SUCCESSFUL"],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

##  Improve model attempt 1!

In [12]:
#The number of hidden neurons should be between the size of the input layer and the size of the output layer.
#The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer.
#The number of hidden neurons should be less than twice the size of the input layer.

In [13]:
#Try a different number of nodes
#Find a new number to try for hidden node. (2/3 inputs)
(len(X_train[0]))/3*2


28.666666666666668

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  29
hidden_nodes_layer2 = 16


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 29)                1276      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                480       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,773
Trainable params: 1,773
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_op1/",exist_ok=True)
checkpoint_path = "checkpoints_op1/weights.{epoch:02d}.hdf5"


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 836us/step - loss: 0.6154 - accuracy: 0.6820
Epoch 2/100
804/804 [==============================] - 1s 838us/step - loss: 0.5571 - accuracy: 0.7262
Epoch 3/100
804/804 [==============================] - 1s 838us/step - loss: 0.5571 - accuracy: 0.7288
Epoch 4/100
804/804 [==============================] - 1s 844us/step - loss: 0.5479 - accuracy: 0.7333
Epoch 5/100
804/804 [==============================] - 1s 830us/step - loss: 0.5533 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 1s 827us/step - loss: 0.5504 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 833us/step - loss: 0.5468 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 836us/step - loss: 0.5432 - accuracy: 0.7382
Epoch 9/100
804/804 [==============================] - 1s 822us/step - loss: 0.5524 - accuracy: 0.7237
Epoch 10/100
804/804 [==============================] - 1s 836us/step - l

804/804 [==============================] - 1s 747us/step - loss: 0.5383 - accuracy: 0.7378
Epoch 80/100
804/804 [==============================] - 1s 815us/step - loss: 0.5321 - accuracy: 0.7417
Epoch 81/100
804/804 [==============================] - 1s 893us/step - loss: 0.5375 - accuracy: 0.7394
Epoch 82/100
804/804 [==============================] - 1s 831us/step - loss: 0.5387 - accuracy: 0.7371
Epoch 83/100
804/804 [==============================] - 1s 744us/step - loss: 0.5401 - accuracy: 0.7313
Epoch 84/100
804/804 [==============================] - 1s 856us/step - loss: 0.5410 - accuracy: 0.7337
Epoch 85/100
804/804 [==============================] - 1s 796us/step - loss: 0.5377 - accuracy: 0.7402
Epoch 86/100
804/804 [==============================] - 1s 802us/step - loss: 0.5429 - accuracy: 0.7364
Epoch 87/100
804/804 [==============================] - 1s 916us/step - loss: 0.5384 - accuracy: 0.7358
Epoch 88/100
804/804 [==============================] - 1s 813us/step - loss:

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5553 - accuracy: 0.7310
Loss: 0.5553300380706787, Accuracy: 0.7309620976448059


In [19]:
#This model is no more accurate
nn.save("AlphabetSoupCharity_2.h5")


# Impove model attempt 2!

In [20]:
# Change activication function for  "Selu". (https://www.tensorflow.org/api_docs/python/tf/keras/activations)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="selu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 352       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_op2/",exist_ok=True)
checkpoint_path = "checkpoints_op2/weights.{epoch:02d}.hdf5"

In [23]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [24]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 1s 663us/step - loss: 0.6469 - accuracy: 0.65530s - loss: 0.6490 - accuracy: 0.65
Epoch 2/100
804/804 [==============================] - 1s 713us/step - loss: 0.5697 - accuracy: 0.7221
Epoch 3/100
804/804 [==============================] - 1s 687us/step - loss: 0.5643 - accuracy: 0.7263
Epoch 4/100
804/804 [==============================] - 1s 827us/step - loss: 0.5624 - accuracy: 0.7267
Epoch 5/100
804/804 [==============================] - 1s 889us/step - loss: 0.5550 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 1s 686us/step - loss: 0.5568 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 1s 688us/step - loss: 0.5531 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 1s 779us/step - loss: 0.5575 - accuracy: 0.7265
Epoch 9/100
804/804 [==============================] - 1s 879us/step - loss: 0.5544 - accuracy: 0.7285
Epoch 10/100
804/804 [=================

804/804 [==============================] - 1s 802us/step - loss: 0.5427 - accuracy: 0.7340
Epoch 80/100
804/804 [==============================] - 1s 767us/step - loss: 0.5448 - accuracy: 0.7322
Epoch 81/100
804/804 [==============================] - 1s 812us/step - loss: 0.5495 - accuracy: 0.7281
Epoch 82/100
804/804 [==============================] - 1s 752us/step - loss: 0.5495 - accuracy: 0.7303
Epoch 83/100
804/804 [==============================] - 1s 809us/step - loss: 0.5399 - accuracy: 0.7392
Epoch 84/100
804/804 [==============================] - 1s 812us/step - loss: 0.5502 - accuracy: 0.7291
Epoch 85/100
804/804 [==============================] - 1s 815us/step - loss: 0.5466 - accuracy: 0.7324
Epoch 86/100
804/804 [==============================] - 1s 723us/step - loss: 0.5455 - accuracy: 0.7318
Epoch 87/100
804/804 [==============================] - 1s 731us/step - loss: 0.5439 - accuracy: 0.7328
Epoch 88/100
804/804 [==============================] - 1s 698us/step - loss:

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5543 - accuracy: 0.7287
Loss: 0.554254949092865, Accuracy: 0.7287463545799255


In [26]:
#Still not 75%!
nn2.save("AlphabetSoupCharity_3.h5")

# Impove model attempt 3!

In [27]:
#Create more bins.

In [28]:
#Investigate Income Amount column?
application_df["INCOME_AMT"].describe()

count     34299
unique        9
top           0
freq      24388
Name: INCOME_AMT, dtype: object

In [29]:
application_df.INCOME_AMT.unique()

array(['0', '1-9999', '10000-24999', '100000-499999', '10M-50M',
       '25000-99999', '50M+', '1M-5M', '5M-10M'], dtype=object)

In [30]:
# See if any values could be binned.
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [31]:
# Binning Values above 5M
replace_income = [ "50M+", "10M-50M", "5M-10M"]
# Replace in dataframe
for income in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(income,"Other")
application_df 

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [32]:
# Generate our categorical variable lists
application_cat2 = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [33]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list

encode2_df =pd.DataFrame(enc.fit_transform(application_df[application_cat2]))
# Add the encoded variable names to the dataframe
encode2_df.columns = enc.get_feature_names(application_cat2)

In [34]:
# Merge one-hot encoded features and drop the originals
app2_df=application_df.merge(encode2_df,left_index=True,right_index=True).drop(application_cat2,1)
app2_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [35]:
# Split our preprocessed data into our features and target arrays
y = app2_df["IS_SUCCESSFUL"].values
X =app2_df.drop(["IS_SUCCESSFUL"],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 336       
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 6         
Total params: 387
Trainable params: 387
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_op3/",exist_ok=True)
checkpoint_path = "checkpoints_op3/weights.{epoch:02d}.hdf5"

In [39]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [40]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 676us/step - loss: 0.6572 - accuracy: 0.6041
Epoch 2/100
804/804 [==============================] - 1s 747us/step - loss: 0.5788 - accuracy: 0.7241
Epoch 3/100
804/804 [==============================] - 1s 759us/step - loss: 0.5614 - accuracy: 0.7274
Epoch 4/100
804/804 [==============================] - 1s 738us/step - loss: 0.5526 - accuracy: 0.7350
Epoch 5/100
804/804 [==============================] - 1s 971us/step - loss: 0.5597 - accuracy: 0.7246
Epoch 6/100
804/804 [==============================] - 1s 893us/step - loss: 0.5585 - accuracy: 0.7273
Epoch 7/100
804/804 [==============================] - 1s 721us/step - loss: 0.5510 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 1s 737us/step - loss: 0.5536 - accuracy: 0.7274
Epoch 9/100
804/804 [==============================] - 1s 909us/step - loss: 0.5508 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 1s 698us/step - l

804/804 [==============================] - 1s 712us/step - loss: 0.5418 - accuracy: 0.7387
Epoch 80/100
804/804 [==============================] - 1s 782us/step - loss: 0.5438 - accuracy: 0.7342
Epoch 81/100
804/804 [==============================] - 1s 684us/step - loss: 0.5437 - accuracy: 0.7332
Epoch 82/100
804/804 [==============================] - 1s 678us/step - loss: 0.5449 - accuracy: 0.7363
Epoch 83/100
804/804 [==============================] - 1s 690us/step - loss: 0.5459 - accuracy: 0.7343
Epoch 84/100
804/804 [==============================] - 1s 679us/step - loss: 0.5421 - accuracy: 0.7347
Epoch 85/100
804/804 [==============================] - 1s 693us/step - loss: 0.5432 - accuracy: 0.7348
Epoch 86/100
804/804 [==============================] - 1s 688us/step - loss: 0.5456 - accuracy: 0.7349
Epoch 87/100
804/804 [==============================] - 1s 682us/step - loss: 0.5420 - accuracy: 0.7357
Epoch 88/100
804/804 [==============================] - 1s 683us/step - loss:

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5539 - accuracy: 0.7318
Loss: 0.553862988948822, Accuracy: 0.7317784428596497


In [42]:
#This model is still not 75% accurate
nn3.save("AlphabetSoupCharity_4.h5")